# 1. Setup and Libraries



In [40]:
# Import TensorFlow and required modules
import tensorflow as tf

# Import Keras layers and models for building the ProGAN
from tensorflow.keras import layers, models, optimizers

# Import NumPy for data handling
import numpy as np

# Import Matplotlib for visualizing generated images
import matplotlib.pyplot as plt

# Import OS for saving/loading models or generated images
import os


# 2. Key Components of ProGAN


**a. The Generator**

In [41]:
# Define the generator model
def build_generator(latent_dim, channels):
    """
    Build the initial generator model for 4x4 resolution.
    latent_dim: Size of the input noise vector
    channels: Number of filters in the initial convolutional layer
    """
    # Define the input layer for the latent vector (noise)
    inputs = layers.Input(shape=(latent_dim,))

    # First dense layer to transform the noise into a 4x4xChannels feature map
    x = layers.Dense(4 * 4 * channels)(inputs)

    # Reshape the dense layer output into 4x4 spatial dimensions
    x = layers.Reshape((4, 4, channels))(x)

    # Apply LeakyReLU activation for non-linearity
    x = layers.LeakyReLU(0.2)(x)

    # Build and return the initial generator model
    model = models.Model(inputs, x)
    return model


# Define a function to progressively add blocks to the generator
def add_generator_block(generator, filters):
    """
    Add a higher-resolution block to the generator.
    generator: Existing generator model
    filters: Number of filters for the new convolutional layers
    """
    # Retain the original input of the generator
    inputs = generator.input

    # Get the current output of the generator
    x = generator.output

    # Upsample the current resolution (e.g., from 4x4 to 8x8)
    x = layers.UpSampling2D()(x)

    # Add a convolutional layer to learn finer details
    x = layers.Conv2D(filters, kernel_size=3, padding="same")(x)

    # Apply LeakyReLU activation for non-linearity
    x = layers.LeakyReLU(0.2)(x)

    # Add a final convolutional layer to map to RGB output (3 channels)
    rgb_output = layers.Conv2D(3, kernel_size=1, padding="same", activation="tanh")(x)

    # Build the updated generator model with the new block
    model = models.Model(inputs, rgb_output)
    return model


**b. The Discriminator**

In [42]:
# Define the discriminator model
def build_discriminator(channels):
    """
    Build the initial discriminator model for 4x4 resolution.
    channels: Number of filters in the initial convolutional layer
    """
    # Define the input layer for images of resolution 4x4 with 3 channels (RGB)
    inputs = layers.Input(shape=(4, 4, 3))

    # First convolutional layer to extract features
    x = layers.Conv2D(channels, kernel_size=3, padding="same")(inputs)

    # Apply LeakyReLU activation for non-linearity
    x = layers.LeakyReLU(0.2)(x)

    # Flatten the feature map for the dense output layer
    x = layers.Flatten()(x)

    # Dense layer to predict real (1) or fake (0)
    x = layers.Dense(1, activation="sigmoid")(x)

    # Build and return the initial discriminator model
    model = models.Model(inputs, x)
    return model


# Define a function to progressively add blocks to the discriminator
def add_discriminator_block(discriminator, filters):
    """
    Add a higher-resolution block to the discriminator.
    discriminator: Existing discriminator model
    filters: Number of filters for the new convolutional layers
    """
    # Define a new input layer for higher-resolution images
    inputs = layers.Input(shape=(None, None, 3))  # Input accepts variable resolution

    # Downsample the input from higher resolution to lower resolution
    x = layers.AveragePooling2D(pool_size=(2, 2))(inputs)

    # Pass the downsampled input to the existing discriminator
    x = discriminator(x)

    # Build and return the updated discriminator model
    model = models.Model(inputs, x)
    return model



# 3. Training Process


```a. Progressive Growth```


In [43]:
# Define the progressive growth of ProGAN
def grow_progan(generator, discriminator, latent_dim, resolutions, filters, epochs_per_stage):
    """
    Gradually grow the ProGAN by adding layers to the generator and discriminator.
    generator: Generator model
    discriminator: Discriminator model
    latent_dim: Size of the input noise vector
    resolutions: List of target resolutions (e.g., [4, 8, 16, 32])
    filters: List of filters for each resolution
    epochs_per_stage: Number of epochs to train at each resolution
    """
    # Iterate through the target resolutions and filters
    for resolution, filter_count in zip(resolutions, filters):
        # Update the generator with a new block
        generator = add_generator_block(generator, filter_count)

        # Update the discriminator with a new block
        discriminator = add_discriminator_block(discriminator, filter_count)

        # Prepare to train models at the current resolution
        print(f"Training at resolution: {resolution}x{resolution}")
        train(generator, discriminator, latent_dim, resolution, epochs_per_stage)


```b. Loss Functions and Optimizers```

In [44]:
# Define the generator loss function
def generator_loss(fake_output):
    """
    Calculates the loss for the generator.
    fake_output: Discriminator predictions for fake images
    """
    return tf.keras.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output)


# Define the discriminator loss function
def discriminator_loss(real_output, fake_output):
    """
    Calculates the loss for the discriminator.
    real_output: Discriminator predictions for real images
    fake_output: Discriminator predictions for fake images
    """
    # Loss for real images (should output 1)
    real_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(real_output), real_output)

    # Loss for fake images (should output 0)
    fake_loss = tf.keras.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output)

    # Return the total discriminator loss
    return real_loss + fake_loss


# Define Adam optimizers for both generator and discriminator
generator_optimizer = optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99)
discriminator_optimizer = optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99)


```c. Training Loop```

In [45]:
# Define a single training step for ProGAN
@tf.function
def train_step(generator, discriminator, images, latent_dim, resolution):
    """
    Executes one step of training for the generator and discriminator.
    generator: Generator model
    discriminator: Discriminator model
    images: Real images from the dataset
    latent_dim: Size of the input noise vector
    resolution: Current resolution for training
    """
    # Resize real images to the current resolution
    images = tf.image.resize(images, (resolution, resolution))

    # Generate random noise vectors for the generator
    noise = tf.random.normal([images.shape[0], latent_dim])

    # Use GradientTape to calculate gradients for both models
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake images using the generator
        fake_images = generator(noise, training=True)

        # Discriminator predictions for real and fake images
        real_output = discriminator(images, training=True)
        fake_output = discriminator(fake_images, training=True)

        # Compute the generator and discriminator losses
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # Calculate gradients for the generator and discriminator
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Apply gradients to update the generator's weights
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    # Apply gradients to update the discriminator's weights
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    # Return the generator and discriminator losses for logging
    return gen_loss, disc_loss


```Training Function```


In [46]:
# Define the training loop for ProGAN
def train(generator, discriminator, latent_dim, resolution, epochs):
    """
    Trains the generator and discriminator at the given resolution.
    generator: Generator model
    discriminator: Discriminator model
    latent_dim: Size of the input noise vector
    resolution: Target resolution for the current training stage
    epochs: Number of epochs to train at the current resolution
    """
    # Load CIFAR-10 dataset
    (x_train, _), _ = tf.keras.datasets.cifar10.load_data()

    # Normalize pixel values to [-1, 1]
    x_train = (x_train.astype("float32") - 127.5) / 127.5

    # Resize images to the target resolution
    x_train = tf.image.resize(x_train, [resolution, resolution]) # Resize images to the target resolution

    # Convert to a TensorFlow dataset for efficient batching
    dataset = tf.data.Dataset.from_tensor_slices(x_train).batch(64)

    # Train for the specified number of epochs
    for epoch in range(epochs):
        for batch in dataset:  # Loop through each batch in the dataset
            # Perform a single training step at the current resolution
            g_loss, d_loss = train_step(generator, discriminator, batch, latent_dim, resolution)

        # Print loss metrics after each epoch
        print(f"Epoch {epoch + 1}, Generator Loss: {g_loss.numpy()}, Discriminator Loss: {d_loss.numpy()}")

# 4. Generate Samples

In [47]:
# Function to generate and visualize samples
def generate_samples(generator, latent_dim, num_samples=16):
    """
    Generates and displays a grid of images using the trained generator.
    generator: Trained generator model
    latent_dim: Size of the input noise vector
    num_samples: Number of images to generate
    """
    # Generate random noise vectors
    noise = tf.random.normal([num_samples, latent_dim])

    # Generate fake images using the generator
    samples = generator(noise, training=False)

    # Denormalize the pixel values to the range [0, 1]
    samples = (samples + 1) / 2

    # Plot the images in a grid
    plt.figure(figsize=(10, 10))
    for i in range(num_samples):
        plt.subplot(4, 4, i + 1)  # Create a 4x4 grid
        plt.imshow(samples[i])
        plt.axis("off")  # Hide axes
    plt.show()


# 5. Putting It All Together



In [48]:
# Define the latent dimension for the noise vector
latent_dim = 128

# Define the number of filters for the initial layers
initial_channels = 64

# Define the progressive resolutions and corresponding filters
resolutions = [4, 8, 16, 32, 64]
filters = [64, 128, 256, 512, 512]  # Filters for each resolution

# Define the number of epochs per stage of resolution
epochs_per_stage = 5

# Build the initial generator and discriminator for 4x4 resolution
generator = build_generator(latent_dim, initial_channels)
discriminator = build_discriminator(initial_channels)

# Train the ProGAN by progressively growing the generator and discriminator
grow_progan(generator, discriminator, latent_dim, resolutions, filters, epochs_per_stage)

# Save the trained generator and discriminator models
generator.save("progan_generator.h5")
discriminator.save("progan_discriminator.h5")

# Generate and visualize samples from the trained generator
generate_samples(generator, latent_dim)


Training at resolution: 4x4


ValueError: in user code:

    File "<ipython-input-35-8bd595d9e467>", line 24, in train_step  *
        real_output = discriminator(images, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/layers/input_spec.py", line 245, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered when calling Functional.call().
    
    [1mInput 0 of layer "functional_18" is incompatible with the layer: expected shape=(None, 4, 4, 3), found shape=(64, 2, 2, 3)[0m
    
    Arguments received by Functional.call():
      • inputs=tf.Tensor(shape=(64, 4, 4, 3), dtype=float32)
      • training=True
      • mask=None
